In [1]:
import pandas as pd
from load_data import Balance, Instruments, AdvisedPortfolios, PriceDB, Singleton

## 추천받은 포트폴리오 -> 잔고(detail 테이블)

In [2]:
instruments_m = pd.read_pickle('../../data/processed/instruments_m.pkl')
price_db = pd.read_pickle('../../data/external/price_db_d.pkl')
advised_pf = pd.read_pickle('../../data/processed/advised_portfolios.pkl')    

In [3]:
userid = 'A00'
username = '투자자0'
current_date = '2021-02-01'
risk_profile = 2

# c_date: 추천 포트폴리오DB에서 사용자가 입력한 날짜와 가장 가까운 날짜.
c_date = advised_pf.loc[advised_pf.date <= current_date, ['date']].max().date
new_units = {'A122260': 2, 'A114260': 4, 'A130730': 2, 'A329650': 13, 'A266370': 4}
prices = {'A122260': [101185.0, 'KRW'], 'A114260': [57900.0, 'KRW'], 'A130730': [100975.0, 'KRW'], 'A329650': [10980.0, 'KRW'], 'A266370': [21325.0, 'KRW']}
df = advised_pf.loc[(advised_pf.date==c_date) & (advised_pf.risk_profile==risk_profile), :]
remaining_cash = 135786

In [4]:
# 보유수량
detail = pd.DataFrame.from_dict(new_units, orient='index').rename(columns={0:'quantity'})
detail.index.name='itemcode'

# 매입가격
temp = pd.DataFrame.from_dict(prices, orient='index').rename(columns={0:'cost_price'}).drop([1], axis=1)
temp.index.name='itemcode'

# 병합
detail = detail.merge(temp, left_index=True, right_index=True, how='inner')

detail['price'] = detail['cost_price']
detail['cost_value'] = detail['quantity']*detail['cost_price']
detail['value'] = detail['quantity']*detail['cost_price']  # 매입가와 평가가격 동일하다고 가정
detail = detail.merge(instruments_m.loc[:, ['itemcode', 'itemname', 'asset_class']], left_on='itemcode', right_on='itemcode', how='left')
detail = detail.reset_index(drop=True)

In [5]:
df_cash = {
    'itemcode': 'C000001',
    'quantity': remaining_cash,
    'cost_price': 1,
    'cost_value': remaining_cash,
    'price':1
    'value': remaining_cash,
    'itemname': '현금',
    'asset_class': 'Cash'
}
df_cash = pd.DataFrame.from_dict(df_cash, orient='index').T

In [6]:
detail = pd.concat([detail, df_cash])

In [7]:
detail['date'] = c_date
detail['date'] = pd.to_datetime(detail['date'], format='%Y-%m-%d').dt.strftime('%m/%d/%Y 4:0:00 PM').astype(str)
detail['userid'] = userid
detail['username'] = username
detail['original'] = 'N'
detail['group_by'] = ''

In [8]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,
0,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,


In [9]:
detail['wt'] = detail.value.transform(lambda x: x/x.sum())

In [10]:
detail = detail.reset_index(drop=True)

In [205]:
col_order = ['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value', 'itemname', 'asset_class', 'date', 'userid', 'username', 'group_by', 'original', 'wt']
detail = detail.loc[:, col_order]

In [196]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217
5,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582


In [200]:
detail.columns

Index(['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value',
       'itemname', 'asset_class', 'date', 'userid', 'username', 'original',
       'group_by', 'wt'],
      dtype='object')

In [203]:
col_order = ['itemcode', 'quantity', 'cost_price', 'price', 'cost_value', 'value', 'itemname', 'asset_class', 'date', 'userid', 'username', 'group_by', 'original', 'wt']

In [210]:
for idx, row in detail.iterrows():
    print(row[col_order].values.tolist())
    print(type(row))

['A122260', 2, 101185.0, 101185.0, 202370.0, 202370.0, 'KOSEF 통안채1년', 'Fixed Income', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.20242141503942002]
<class 'pandas.core.series.Series'>
['A114260', 4, 57900.0, 57900.0, 231600.0, 231600.0, 'KODEX 국고채3년', 'Fixed Income', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.2316588413457018]
<class 'pandas.core.series.Series'>
['A130730', 2, 100975.0, 100975.0, 201950.0, 201950.0, 'KOSEF 단기자금', 'Fixed Income', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.20200130833231641]
<class 'pandas.core.series.Series'>
['A329650', 13, 10980.0, 10980.0, 142740.0, 142740.0, 'KODEX TRF3070', 'Alternative', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.14277626517135353]
<class 'pandas.core.series.Series'>
['A266370', 4, 21325.0, 21325.0, 85300.0, 85300.0, 'KODEX IT', 'Equity', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.08532167170461297]
<class 'pandas.core.series.Series'>
['C000001', 135786, 1, nan, 135786, 135786, '현금', 'Cash', '02/

In [211]:
for idx, row in detail.iterrows():
    print(row.values.tolist())
    print(type(row))

['A122260', 2, 101185.0, 101185.0, 202370.0, 202370.0, 'KOSEF 통안채1년', 'Fixed Income', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.20242141503942002]
<class 'pandas.core.series.Series'>
['A114260', 4, 57900.0, 57900.0, 231600.0, 231600.0, 'KODEX 국고채3년', 'Fixed Income', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.2316588413457018]
<class 'pandas.core.series.Series'>
['A130730', 2, 100975.0, 100975.0, 201950.0, 201950.0, 'KOSEF 단기자금', 'Fixed Income', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.20200130833231641]
<class 'pandas.core.series.Series'>
['A329650', 13, 10980.0, 10980.0, 142740.0, 142740.0, 'KODEX TRF3070', 'Alternative', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.14277626517135353]
<class 'pandas.core.series.Series'>
['A266370', 4, 21325.0, 21325.0, 85300.0, 85300.0, 'KODEX IT', 'Equity', '02/01/2021 4:0:00 PM', 'A00', '투자자0', '', 'N', 0.08532167170461297]
<class 'pandas.core.series.Series'>
['C000001', 135786, 1, nan, 135786, 135786, '현금', 'Cash', '02/

In [218]:
detail.userid[0]

'A00'

In [217]:
detail.date[0]

0    02/01/2021 4:0:00 PM
1    02/01/2021 4:0:00 PM
2    02/01/2021 4:0:00 PM
3    02/01/2021 4:0:00 PM
4    02/01/2021 4:0:00 PM
5    02/01/2021 4:0:00 PM
Name: date, dtype: object

In [219]:
new_general = detail.loc[:, ['wt', 'value', 'asset_class']].groupby(
                'asset_class').sum().sort_values('wt', ascending=False).reset_index()
new_general['userid'] = detail.userid[0]
new_general['date'] = detail.date[0]


In [220]:
new_general

,asset_class,wt,value,userid,date
0,Fixed Income,0.636082,635920.0,A00,02/01/2021 4:0:00 PM
1,Alternative,0.142776,142740.0,A00,02/01/2021 4:0:00 PM
2,Cash,0.135820,135786.0,A00,02/01/2021 4:0:00 PM
3,Equity,0.085322,85300.0,A00,02/01/2021 4:0:00 PM


## 리밸런싱

In [17]:
from datetime import datetime
current_date

'2021-02-01'

In [18]:
end_date = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].max()
dates = advised_pf.loc[(advised_pf.risk_profile==risk_profile) & (advised_pf.date > current_date), 'date'].unique()

In [190]:
dt = datetime.strptime('2021-02-01', '%Y-%m-%d')

In [192]:
dt = datetime.strptime('2021-02-01', '%Y-%m-%d')
dt = str(dt.month)+'/'+str(dt.day)+'/'+str(dt.year)+' 4:00:00 PM'

In [194]:
dt

'2/1/2021 4:00:00 PM'

In [188]:
dt

'02/01/021 9:00:00 AM'

In [21]:
detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217
5,C000001,135786,1,NaN,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582


In [69]:
dates = advised_pf.loc[(advised_pf.risk_profile == risk_profile) & (
    advised_pf.date > current_date), 'date'].unique()
every5day = dates[::5]

In [70]:
dates

array(['2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05',
       '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11',
       '2021-02-12', '2021-02-15', '2021-02-16', '2021-02-17',
       '2021-02-18', '2021-02-19', '2021-02-22', '2021-02-23',
       '2021-02-24', '2021-02-25', '2021-02-26', '2021-03-01',
       '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05',
       '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11',
       '2021-03-12', '2021-03-15', '2021-03-16', '2021-03-17',
       '2021-03-18', '2021-03-19'], dtype=object)

In [139]:
import copy

In [ ]:
next_detail = copy.deepcopy(detail)
all_the_nexts = pd.DataFrame(columns=next_detail.columns)
for dt in dates:
    prices_dt = price_db.loc[price_db.date==dt, ['price', 'itemcode']].reset_index(drop=True)
    holding_itemcodes = detail.itemcode.to_list()
    holding_prices = prices_dt[prices_dt.itemcode.isin(holding_itemcodes)]
    next_date = datetime.strptime(dt, '%Y-%m-%d')
    next_date = str(next_date.month)+'/'+str(next_date.day)+'/'+str(next_date.year)+' 09:00:00 AM'
    next_detail = copy.deepcopy(next_detail)
    next_detail.merge(holding_prices, left_on='itemcode', right_on='itemcode', how='left', suffixes=('_old', '')).drop('price_old', axis=1)
    next_detail.loc[next_detail.itemcode=='C000001', 'price'] = 1
    next_detail['date'] = next_date
    all_the_nexts = pd.concat((all_the_nexts, next_detail))

In [169]:
all_the_nexts = pd.DataFrame(columns=next_detail.columns)

In [177]:
all_the_nexts = pd.concat((all_the_nexts, next_detail))

In [178]:
all_the_nexts

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217
5,C000001,135786,1,1.0,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776


In [167]:
next_detail.loc[next_detail.itemcode=='C000001', 'price'] = 1

In [168]:
next_detail

,itemcode,quantity,cost_price,price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt
0,A122260,2,101185,101185.0,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421
1,A114260,4,57900,57900.0,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659
2,A130730,2,100975,100975.0,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001
3,A329650,13,10980,10980.0,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776
4,A266370,4,21325,21325.0,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217
5,C000001,135786,1,1.0,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582


In [151]:
dt = datetime.strptime(dates[1], '%Y-%m-%d')

In [152]:
dt2 = str(dt.month)+'/'+str(dt.day)+'/'+str(dt.year)+' 09:00:00 AM'

In [146]:
dt2

'2/2/2021 09:00:00 AM'

In [164]:
next_detail.merge(holding_prices, left_on='itemcode', right_on='itemcode', how='left', suffixes=('_old', '')).drop('price_old', axis=1)

,itemcode,quantity,cost_price,cost_value,value,itemname,asset_class,date,userid,username,original,group_by,wt,price
0,A122260,2,101185,202370,202370,KOSEF 통안채1년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202421,101105.0
1,A114260,4,57900,231600,231600,KODEX 국고채3년,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.231659,58125.0
2,A130730,2,100975,201950,201950,KOSEF 단기자금,Fixed Income,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.202001,100895.0
3,A329650,13,10980,142740,142740,KODEX TRF3070,Alternative,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.142776,NaN
4,A266370,4,21325,85300,85300,KODEX IT,Equity,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.0853217,22020.0
5,C000001,135786,1,135786,135786,현금,Cash,02/01/2021 4:0:00 PM,A00,투자자0,N,,0.13582,NaN


In [133]:
prices_dt = price_db.loc[price_db.date==dt, ['price', 'itemcode']].reset_index(drop=True)

In [129]:
holdings = detail.itemcode.to_list()
holdings

['A122260', 'A114260', 'A130730', 'A329650', 'A266370', 'C000001']

In [156]:
holding_prices = prices_dt[prices_dt.itemcode.isin(holdings)]

In [157]:
holding_prices

,price,itemcode
8,100895.0,A130730
9,101105.0,A122260
23,22020.0,A266370
24,58125.0,A114260


## 자산군별 바 차트

In [28]:
import plotly.express as px

In [43]:
v = detail.loc[:, ['wt', 'itemname']].groupby(
                    'itemname').sum().sort_values('wt', ascending=False).reset_index()

In [29]:
a = detail.loc[:, ['wt', 'asset_class']].groupby(
                    'asset_class').sum().sort_values('wt', ascending=False).reset_index()

In [50]:
asset_class = ['Fixed Income', 'Alternative', 'Cash', 'Equity']

In [53]:
df_ac = pd.DataFrame(asset_class, columns=['asset_class'])

In [62]:
df_by_asset_class = a.merge(df_ac, left_on='asset_class', right_on='asset_class', how='right').sort_values('wt', ascending=False).reset_index()

In [228]:
'A' + ('00' + str(1))[-3:]

'A001'

In [44]:
fig1 = px.pie(v, names = 'itemname', values = 'wt', title = "추천 포트폴리오", color_discrete_sequence=px.colors.qualitative.Set3)
fig1.show()

In [64]:
df_by_asset_class.loc[:, ['asset_class', 'wt']]

,asset_class,wt
0,Fixed Income,0.636082
1,Alternative,0.142776
2,Cash,0.135820
3,Equity,0.085322


In [65]:
fig2 = px.bar(df_by_asset_class, y = 'wt', x = 'asset_class', title = '자산군별 비중',
       labels = {'asset_class': '자산군', 'wt': '비중'},
       orientation = 'v', color = "asset_class", color_continuous_scale = 'darkmint',
        template = 'plotly_dark')
fig2.show()